# Mexican Covid

At the end of 2019, the world saw the born of the Sars-cov-2 virus which genereate a disease called COVID19 (COrona VIrus disease). The high risk that we face with this virus is that the sympthons are related to a typical flu but generate a high damage un the lungs decreasing the oxygen level. The disminution of oxygen generate a high demand of intensive beds.

From WHO files

According to the WHO the incubation period for COVID-19, is on average 5-6 days, but can be extended to 14 days. However there some evidence that shows that exist different types of transmition, such as symptomatic, pre-symptomatic and a symptomatic.

The typical transmition is the symptomatic, this ocurrs when a person shown COVID symptomatic and can spread the virus conciently or inconciently respiratory droplest over thier enviroment. However, some cases has shown that a person can transmit COVID-19 without symptoms, (3 or 2 days) before present the typical symptoms. Finally there are some concern about the asyptomatic transmission according to the WHO: Asymptomatic transmission refers to transmission of the virus from a person, who does not develop symptoms. There are few reports of laboratory-confirmed cases who are truly asymptomatic, and to date, there has been no documente asymptomatic transmission. This does not exclude the possibility that it may occur. Asymptomatic cases have been reported as part of contact tracing efforts in some countries. Source https://www.who.int/docs/default-source/coronaviruse/situation-reports/20200402-sitrep-73-covid-19.pdf?sfvrsn=5ae25bc7_2#:~:text=An%20asymptomatic%20laboratory%2Dconfirmed,more%20information%20becomes%20available.


The goal of this notebook is listed below:
* Understand the evolution of the covi19 in Mexico, there are a rate of 10% of deads though a visualization analysis


Key elements

* Incubation period can be expresed a GAMMA.
* Is most common the transmision after show symptoms.
* How we can modeled (explain pre-symtomatic transmition)
* how we can modeled asyntomatic transmition


In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib 
import glob

Using matplotlib backend: MacOSX


In [2]:
import glob
strpath = '../src/'
m = []
for filename in glob.glob(strpath+"*"):
    print(filename)
    date = filename[7:9] +'-' + filename[9:11] +'-' +filename[11:13]
    df = pd.read_csv(filename, encoding = "ISO-8859-1",dtype =str)
    df['FECHA_ACTUALIZACION1'] = pd.to_datetime(df['FECHA_ACTUALIZACION'],)
    df.FECHA_INGRESO = pd.to_datetime(df['FECHA_INGRESO'],)
    df.FECHA_SINTOMAS = pd.to_datetime(df['FECHA_SINTOMAS'],)
    df['deltaSintomas']=df.FECHA_ACTUALIZACION1-df.FECHA_SINTOMAS 
    df['deltaSintomas'] = df.deltaSintomas.dt.days
    df.loc[df.FECHA_DEF=='9999-99-99', 'FECHA_DEF'] = '2050-12-31'
    df.FECHA_DEF = pd.to_datetime(df['FECHA_DEF'],)
    m.append([df.FECHA_ACTUALIZACION.unique()[0]
              ,df[(df.deltaSintomas<14)& (df.RESULTADO=='1')]['ID_REGISTRO'].count()
          ,df[df.RESULTADO=='1']['ID_REGISTRO'].count()
          ,df[(df.FECHA_DEF!='2050-12-31')&(df.RESULTADO=='1')]['ID_REGISTRO'].count()
          ,df[df.RESULTADO=='3']['ID_REGISTRO'].count()
          ,df.ID_REGISTRO.nunique()
          ,len(df[(df.deltaSintomas<14)& (df.RESULTADO=='3')])
          ,len(df[(df.FECHA_DEF<'2020-12-31')&(df.RESULTADO=='3')])
          ,len(df[(df.FECHA_DEF<'2020-12-31')&(df.RESULTADO=='2')])
             ]
    )
df = pd.DataFrame(m)
df.columns = ['date', 'activos', 'positivos', 'muertes', 'sospechosos', 'muestra', 'activos sospechosos', 'defunciones sospechosos', 'muertos negativos']


../src/20200604.csv
../src/20200610.csv
../src/20200412.csv
../src/20200413.csv
../src/20200611.csv
../src/20200605.csv
../src/20200613.csv
../src/20200607.csv
../src/20200606.csv
../src/20200612.csv
../src/20200616.csv
../src/20200602.csv
../src/20200414.csv
../src/20200428.csv
../src/20200429.csv
../src/20200415.csv
../src/20200603.csv
../src/20200617.csv
../src/20200601.csv
../src/20200615.csv
../src/20200417.csv
../src/20200416.csv
../src/20200614.csv
../src/20200511.csv
../src/20200505.csv
../src/20200504.csv
../src/20200510.csv
../src/20200506.csv
../src/20200512.csv
../src/20200513.csv
../src/20200507.csv
../src/20200503.csv
../src/20200517.csv
../src/20200516.csv
../src/20200502.csv
../src/20200514.csv
../src/20200528.csv
../src/20200529.csv
../src/20200501.csv
../src/20200515.csv
../src/20200518.csv
../src/20200530.csv
../src/20200524.csv
../src/20200525.csv
../src/20200531.csv
../src/20200519.csv
../src/20200527.csv
../src/20200526.csv
../src/20200522.csv
../src/20200523.csv


In [3]:
df = df.sort_values(by='date', ascending=False)
df = df.reset_index(drop=True)
df

,date,activos,positivos,muertes,sospechosos,muestra,activos sospechosos,defunciones sospechosos,muertos negativos
0,2020-06-24,24036,196847,24324,62475,515658,40219,1894,6412
1,2020-06-23,24387,191410,23377,59106,501871,37894,1848,6260
2,2020-06-22,23155,185122,22584,57281,488550,35710,1874,6111
3,2020-06-21,24225,180545,21825,56590,479528,35875,1892,6035
4,2020-06-20,23567,175202,20781,60621,473952,40333,1895,5990
...,...,...,...,...,...,...,...,...,...
68,2020-04-19,627,5399,406,10792,41593,5664,90,359
69,2020-04-19,2955,8261,686,10139,49570,7291,112,490
70,2020-04-19,2273,7497,650,12369,49167,9077,123,478
71,2020-04-19,918,5847,449,11717,44436,6899,92,396


In [4]:
df.transpose().to_csv('newfile.csv',)

In [34]:
frames = [df1, df2]
df = pd.concat(frames)

In [35]:
df.FECHA_ACTUALIZACION = pd.to_datetime(df['FECHA_ACTUALIZACION'],)
df.FECHA_INGRESO = pd.to_datetime(df['FECHA_INGRESO'],)
df.FECHA_SINTOMAS = pd.to_datetime(df['FECHA_SINTOMAS'],)
df['deltaSintomas']=df.FECHA_ACTUALIZACION-df.FECHA_SINTOMAS
df['deltaSintomas'] = df.deltaSintomas.dt.days
df.loc[df.FECHA_DEF=='9999-99-99', 'FECHA_DEF'] = '2050-12-31'
df.FECHA_DEF = pd.to_datetime(df['FECHA_DEF'],)
m=[]
m.append([df[(df.deltaSintomas<14)& (df.RESULTADO=='1')]['ID_REGISTRO'].count(),
          df[df.RESULTADO=='1']['ID_REGISTRO'].count()
          ,df[df.FECHA_DEF>'2050-12-31']['ID_REGISTRO'].count()
          ,df[df.RESULTADO=='3']['ID_REGISTRO'].count()
          ,df.ID_REGISTRO.nunique()])

In [36]:
df.FECHA_ACTUALIZACION.unique()

array(['2020-06-21T00:00:00.000000000', '2020-06-20T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [6]:
m.append([date,df[(df.deltaSintomas<14)& (df.RESULTADO==1)]['ID_REGISTRO'].count(),
          df[df.RESULTADO==1]['ID_REGISTRO'].count()
          ,df[df.FECHA_DEF!='2050-12-31']['ID_REGISTRO'].count()
          ,df[df.RESULTADO==3]['ID_REGISTRO'].count()
          ,df.ID_REGISTRO.nunique()])

In [8]:
len(df[(df.FECHA_DEF<'2020-06-22')])-len(df[(df.FECHA_DEF<'2020-06-22')&(df.RESULTADO!='3')])

1892

In [28]:
print('Activos Sospechosos %d, ∆ %d' % (len(df[(df.deltaSintomas<14)& (df.RESULTADO=='3')]),1))

Activos 1, ∆ 1


35875

1892

6035